In [ ]:
import pandas as pd
df_restaurants = pd.read_csv("Final_restaurants.csv")
df_users = pd.read_csv("final_users.csv")
df_reviews = pd.read_csv("final_reviews.csv")

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

# Initialize the tokenizer and model for embeddings
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def get_embeddings(texts: list) -> list:
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy().tolist()

df_restaurants["cuisine_embeddings"] = get_embeddings(df_restaurants["cuisine"].tolist())
df_users["cuisine_preference_embeddings"] = get_embeddings(df_users["cuisine_preference"].tolist())

# Get embeddings for reviews
df_reviews["review_embeddings"] = get_embeddings(df_reviews["review"].tolist())

In [ ]:
import clickhouse_connect

client = clickhouse_connect.get_client(
    host='msc-b3eeeb4c.us-east-1.aws.myscale.com',
    port=443,
    username='usamajamil3434_org_default',
    password='passwd_qOpNExYtVW2OcW'
)

In [ ]:

client.command("""
    CREATE TABLE default.users (
        userId Int64,
        cuisine_preference String,
        rating_preference Float32,
        price_range Int64,
        latitude Float32,
        longitude Float32,
        cuisine_preference_embeddings Array(Float32),
        CONSTRAINT check_data_length CHECK length(cuisine_preference_embeddings) = 384
    ) ENGINE = MergeTree()
    ORDER BY userId
    """)

client.command("""
    CREATE TABLE default.reviews (
        userId Int64,
        restaurantId Int64,
        rating Float32,
        review String,
        review_embeddings Array(Float32),
        CONSTRAINT check_data_length CHECK length(review_embeddings) = 384
    ) ENGINE = MergeTree()
    ORDER BY userId
    """)


client.command("""
    CREATE TABLE default.restaurants (
        restaurantId Int64,
        name String,
        cuisine String,
        rating Float32,
        price_range Int64,
        latitude Float32,
        longitude Float32,
        cuisine_embeddings Array(Float32),
        CONSTRAINT check_data_length CHECK length(cuisine_embeddings) = 384
    ) ENGINE = MergeTree()
    ORDER BY restaurantId
    """)


In [ ]:
client.insert("default.users", df_users.to_records(index=False).tolist(), column_names=df_users.columns.tolist())
client.insert("default.reviews", df_reviews.to_records(index=False).tolist(), column_names=df_reviews.columns.tolist())
client.insert("default.restaurants", df_restaurants.to_records(index=False).tolist(), column_names=df_restaurants.columns.tolist())

In [ ]:
client.command("""
ALTER TABLE default.users
    ADD VECTOR INDEX user_index cuisine_preference_embeddings
    TYPE MSTG
""")
client.command("""
ALTER TABLE default.restaurants
    ADD VECTOR INDEX restaurant_index cuisine_embeddings
    TYPE MSTG
""")

client.command("""
ALTER TABLE default.reviews
    ADD VECTOR INDEX reviews_index review_embeddings
    TYPE MSTG
""")